In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 15.3 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 62.9 MB/s 
     |████████████████████████████████| 6.5 MB 93.9 MB/s 
     |████████████████████████████████| 596 kB 86.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import os
from transformers import AutoTokenizer, GPT2LMHeadModel

# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# import gluonnlp as nlp
# from gluonnlp.data import SentencepieceTokenizer, SentencepieceDetokenizer
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import re

import random
from random import sample

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
     bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
    )


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

In [6]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer

# torch version
class GPT2FewshotClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # self.gpt2 = GPT2LMHeadModel.from_pretrained(dir_path, from_tf=True)
        # self.gpt = GPT2LMHeadModel.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
        # self.gpt2 = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
        tokenizer = AutoTokenizer.from_pretrained(
            'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
            bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
            )
        self.gpt = AutoModelForCausalLM.from_pretrained(
            'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
            pad_token_id=tokenizer.eos_token_id, 
            torch_dtype='auto', low_cpu_mem_usage=True).to(device='cuda', non_blocking=True)
    
    def forward(self, x):
        # import pdb; pdb.set_trace()

        outputs = self.gpt(x)[0][:, -1, :]
        # outputs = self.gpt2(x)
        
        return outputs

In [7]:
import pandas as pd
tr_path = '/content/drive/MyDrive/few_shot/data/ratings_train.csv'
tst_path = '/content/drive/MyDrive/few_shot/data/ratings_test.csv'
train_data = pd.read_csv(tr_path).dropna()
train_fewshot_data = []

for train_sent, train_label in train_data[['document', 'label']].values:
    # tokens = vocab[tokenizer(train_sent)]
    tokens = tokenizer(train_sent)['input_ids']

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))



In [8]:
test_data = pd.read_csv(tst_path).dropna()

sample_size = 1000
train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 30)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=42)

In [9]:
model = GPT2FewshotClassifier().cuda()

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# def build_prompt_text(sent):
#     return "문장: " + sent + ' 감정: '


def build_prompt_text(sent):
    return '감정 분석 문장: ' + sent + ' 결과: '


def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

real_labels = []
pred_tokens = []

for i, (test_sent, test_label) in enumerate(tqdm(test_data[['document', 'label']].values)):
    tokens = tokenizer('<s>')['input_ids']

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += '긍정\n' if example_label == 1 else '부정\n'

        tokens += tokenizer(appended_prompt_example_text)['input_ids']

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)
    
    test_tokens = tokenizer(appended_prompt_sent)['input_ids']
    
    tokens += test_tokens
    model.eval()
    model = model.cuda()

    with torch.no_grad():

        # x = tokenizer.encode(hf_tokens)
        outputs = model(torch.tensor([tokens]).cuda())
        pred = torch.argmax(outputs, axis=-1)

        pred_tokens.append(pred.cpu().item())

    pos = tokenizer(' 긍정')['input_ids']
    negative = tokenizer(' 부정')['input_ids']            
    
    label = pos if test_label == 1 else negative
      
    real_labels.append(label[0])
    

 32%|███▏      | 315/1000 [00:50<01:46,  6.43it/s]

In [ ]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)